In [1]:
import allensdk_lims.internal.core.lims_utilities as lu
from allensdk.core.nwb_data_set import NwbDataSet
from allensdk.ephys.extract_cell_features import extract_cell_features

import os
import numpy as np
import pandas as pd
from collections import defaultdict



%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#SQL query from David
query = """
select wkf.storage_directory || wkf.filename as nwb_path, sp.id as specimen_id, sp.name as specimen_name
from well_known_files wkf 
join ephys_roi_results err on wkf.attachable_id = err.id
join specimens sp on sp.ephys_roi_result_id = err.id
where wkf.well_known_file_type_id = 475137571
and err.workflow_state = 'manual_passed'

"""
nwb_files = lu.query(query)
cell_index = { c['specimen_id']: c for c in nwb_files}

nwb_df = pd.DataFrame(nwb_files)

query2 = """
select es.sweep_number, est.description as stimulus_code, estt.name as stimulus_type from ephys_sweeps es
join ephys_stimuli est on es.ephys_stimulus_id = est.id
join ephys_stimulus_types estt on estt.id = est.ephys_stimulus_type_id
where es.specimen_id = """

custom_report = pd.read_csv('custom_report.csv')
v1_df = custom_report
v1_df = v1_df[v1_df['ephys_qc'] == 'manual_passed']

new_columns = ['tau','sag','ri','sag_amp','ISI_initial/ISI_mean','adaptation_index',
               'capacitance','AP_amp','upstroke_downstroke_ratio','v_rest',
               'rheobase_sweep','latency','instant_sweep','rheobase_i', 
               'noise_1','noise_2','hero_rate','fast_trough','hero_sweep']

for i in new_columns:
    v1_df[i] = None

for i,cell in enumerate(v1_df['cell_specimen_id']):
    v1_df['nwb_filename'].iloc[i] = nwb_df['nwb_path'][nwb_df.specimen_id == cell].iloc[0]
    v1_df['nwb_filename'].iloc[i] = lu.linux_to_windows(v1_df['nwb_filename'].iloc[i])
    file_name =  v1_df['nwb_filename'].iloc[i]
    data_set = NwbDataSet(file_name)
    #print file_name
    sweeps = lu.query(query2+str(cell))
    sweep_numbers = defaultdict(list)
    for sweep in sweeps:
        sweep_numbers[sweep['stimulus_type']].append(sweep['sweep_number'])
        
    if sweep_numbers['Long Square'] != []:
        cell_features = extract_cell_features(NwbDataSet(file_name),
                                      sweep_numbers['Ramp'],
                                      sweep_numbers['Short Square'],
                                      sweep_numbers['Long Square'])


        v1_df['tau'].iloc[i] = cell_features['long_squares']['tau']
        v1_df['sag'].iloc[i] = cell_features['long_squares']['sag']
        v1_df['ri'].iloc[i] = cell_features['long_squares']['input_resistance']
        v1_df['sag_amp'].iloc[i] = cell_features['long_squares']['v_baseline'] - (cell_features['long_squares']['vm_for_sag']*cell_features['long_squares']['sag'])
        #alm_df['ISI_initial/ISI_mean'].iloc[i] = cell_features['long_squares']['hero_sweep']['first_isi']/cell_features['long_squares']['hero_sweep']['mean_isi']
        v1_df['adaptation_index'].iloc[i] = cell_features['long_squares']['hero_sweep']['adapt']
        v1_df['capacitance'].iloc[i] = cell_features['long_squares']['tau']/cell_features['long_squares']['input_resistance']*1000
        v1_df['AP_amp'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['peak_v'] - cell_features['long_squares']['hero_sweep']['spikes'][0]['threshold_v']
        v1_df['upstroke_downstroke_ratio'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['spikes'][0]['upstroke_downstroke_ratio']
        v1_df['v_rest'].iloc[i] = cell_features['long_squares']['v_baseline']
        v1_df['rheobase_sweep'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['id']
        v1_df['latency'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['latency']
        v1_df['instant_sweep'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['id']
        v1_df['rheobase_i'].iloc[i] = cell_features['long_squares']['rheobase_i']
        v1_df['hero_rate'].iloc[i] = cell_features['long_squares']['hero_sweep']['avg_rate']
        spikes = cell_features['long_squares']['hero_sweep']['spikes']
        v1_df['fast_trough'].iloc[i] = np.mean([spikes[f]['fast_trough_v'] for f in range(len(spikes))])
        v1_df['hero_sweep'].iloc[i] = cell_features['long_squares']['hero_sweep']['id']

    print cell

for row in range(len(v1_df)):
    for title in v1_df.columns:
        if type(v1_df[title].iloc[row]) == tuple:
            v1_df[title].iloc[row] = v1_df[title].iloc[row][0]

v1_df.to_csv('all_features.csv')

505795846
505798354
505804754


TypeError: slice indices must be integers or None or have an __index__ method

In [42]:
v1_df.to_csv('all_features.csv')


In [ ]:
v1_df = v1_df[v1_df['project'] == 'T301']
v1_df = v1_df[v1_df['cell_specimen_id'] != cell]
for i in range(len(v1_df)):
    if v1_df['hero_rate'].iloc[i] == None:
        cell = v1_df['cell_specimen_id'].iloc[i]
        v1_df['nwb_filename'].iloc[i] = nwb_df['nwb_path'][nwb_df.specimen_id == cell].iloc[0]
        v1_df['nwb_filename'].iloc[i] = lu.linux_to_windows(v1_df['nwb_filename'].iloc[i])
        file_name =  v1_df['nwb_filename'].iloc[i]
        data_set = NwbDataSet(file_name)
        #print file_name
        sweeps = lu.query(query2+str(cell))
        sweep_numbers = defaultdict(list)
        for sweep in sweeps:
            sweep_numbers[sweep['stimulus_type']].append(sweep['sweep_number'])

        if sweep_numbers['Long Square'] != []:
            
            cell_features = extract_cell_features(NwbDataSet(file_name),
                                          sweep_numbers['Ramp'],
                                          sweep_numbers['Short Square'],
                                          sweep_numbers['Long Square'])


            v1_df['tau'].iloc[i] = cell_features['long_squares']['tau']
            v1_df['sag'].iloc[i] = cell_features['long_squares']['sag']
            v1_df['ri'].iloc[i] = cell_features['long_squares']['input_resistance']
            v1_df['sag_amp'].iloc[i] = cell_features['long_squares']['v_baseline'] - (cell_features['long_squares']['vm_for_sag']*cell_features['long_squares']['sag'])
            #alm_df['ISI_initial/ISI_mean'].iloc[i] = cell_features['long_squares']['hero_sweep']['first_isi']/cell_features['long_squares']['hero_sweep']['mean_isi']
            v1_df['adaptation_index'].iloc[i] = cell_features['long_squares']['hero_sweep']['adapt']
            v1_df['capacitance'].iloc[i] = cell_features['long_squares']['tau']/cell_features['long_squares']['input_resistance']*1000
            v1_df['AP_amp'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['peak_v'] - cell_features['long_squares']['hero_sweep']['spikes'][0]['threshold_v']
            v1_df['upstroke_downstroke_ratio'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['spikes'][0]['upstroke_downstroke_ratio']
            v1_df['v_rest'].iloc[i] = cell_features['long_squares']['v_baseline']
            v1_df['rheobase_sweep'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['id']
            v1_df['latency'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['latency']
            v1_df['instant_sweep'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['id']
            v1_df['rheobase_i'].iloc[i] = cell_features['long_squares']['rheobase_i']
            v1_df['hero_rate'].iloc[i] = cell_features['long_squares']['hero_sweep']['avg_rate']
            spikes = cell_features['long_squares']['hero_sweep']['spikes']
            v1_df['fast_trough'].iloc[i] = np.mean([spikes[f]['fast_trough_v'] for f in range(len(spikes))])
            v1_df['hero_sweep'].iloc[i] = cell_features['long_squares']['hero_sweep']['id']

        print cell

for row in range(len(v1_df)):
    for title in v1_df.columns:
        if type(v1_df[title].iloc[row]) == tuple:
            v1_df[title].iloc[row] = v1_df[title].iloc[row][0]

v1_df.to_csv('all_features.csv')

In [2]:
if v1_df['hero_rate'].iloc[500] == None:
    print 'test'

test


In [3]:
v1_df.to_csv('all_features.csv')

In [4]:
for i in range(0,10):
    print i

0
1
2
3
4
5
6
7
8
9
